In [1]:
import json

In [2]:
# 先划分集合
test_querys = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/test_query.jsonl', 'r') as f:
    for line in f:
        query = json.loads(line)
        test_querys[str(query['id'])] = query

common_query = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/common_query.json', 'r') as f:
    for line in f:
        query = json.loads(line)
        if str(query['id']) not in test_querys.keys():
            continue
        common_query[str(query['id'])] = query

controversial_query = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/controversial_query.json', 'r') as f:
    for line in f:
        query = json.loads(line)
        if str(query['id']) not in test_querys.keys():
            continue
        controversial_query[str(query['id'])] = query

expedited_query = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/query/procedural_query.json', 'r') as f:
    for line in f:
        query = json.loads(line)
        if str(query['id']) not in test_querys.keys():
            continue
        if '速裁' not in query['query']:
            continue
        expedited_query[str(query['id'])] = query

In [3]:
# 读入各个模型的排序文件
bm25_result = json.load(open('/share/kelong/chenlong/Legal/Ranking_LeCaRDv2/examples/bm25重排结果/rank_bm25.json', 'r'))
bert_finetune_result = json.load(open('./ranking_results/BERT_LeCaRDv2_best_finetuned.json', 'r'))
SAILER_finetune_result = json.load(open('./ranking_results/SAILER_LeCaRDv2_zeroshot.json', 'r'))
KELLER_finetune_result = json.load(open('./ranking_results/KELLER_LeCaRDv2_best_finetuned.json', 'r'))
# 读入labels，然后扩充一下labels
labels = {}
with open('/share/kelong/chenlong/Legal/data/LeCaRDv2-main/label/relevence.trec', 'r') as f:
    for line in f:
        qid, _, did, rel = line.split('\t')
        if qid not in labels.keys():
            labels[qid] = {}
        labels[qid][did] = int(rel)

for qid in bert_finetune_result.keys():
    for did in bert_finetune_result[qid].keys():
        if did not in labels[qid].keys():
            labels[qid][did] = 0

In [4]:
# 分别测试三个结果在三种query上的指标
from Metrics import compute_metrics_normal
model_results = {'bm25': bm25_result, 'BERT': bert_finetune_result, 'SAILER': SAILER_finetune_result, 'KELLER': KELLER_finetune_result}
query_types = {'common': common_query, 'controversial': controversial_query, 'expedited': expedited_query}
for model_name in model_results.keys():
    for query_type in query_types.keys():
        sub_result = {qid: model_results[model_name][qid] for qid in model_results[model_name].keys() if qid in query_types[query_type].keys()}
        print(f'{model_name}, {query_type}', str(compute_metrics_normal(sub_result, labels)))

bm25, common {'MAP': 0.5439972774497908, 'MRR': 0.6168672173552824, 'P@1': 0.5166666666666667, 'P@3': 0.47777777777777786, 'NDCG@3': 0.7199534738871883, 'NDCG@5': 0.7234943955093239, 'NDCG@10': 0.7412014456941162, 'NDCG@20': 0.7988765509011049, 'NDCG@30': 0.8750150820296415}
bm25, controversial {'MAP': 0.6173286951591025, 'MRR': 0.6694047619047618, 'P@1': 0.55, 'P@3': 0.5166666666666667, 'NDCG@3': 0.7352342631151669, 'NDCG@5': 0.7408125844535962, 'NDCG@10': 0.7605288661341124, 'NDCG@20': 0.8022617111904644, 'NDCG@30': 0.88803138387663}
bm25, expedited {'MAP': 0.9198561088154684, 'MRR': 1.0, 'P@1': 1.0, 'P@3': 1.0, 'NDCG@3': 1.0, 'NDCG@5': 0.9306816431612379, 'NDCG@10': 0.9902658207362975, 'NDCG@20': 0.9926989047702794, 'NDCG@30': 0.9930423019548571}
BERT, common {'MAP': 0.5948524068240777, 'MRR': 0.6783356203981206, 'P@1': 0.575, 'P@3': 0.5361111111111111, 'NDCG@3': 0.7802700279830772, 'NDCG@5': 0.7866639124521521, 'NDCG@10': 0.8045331611469031, 'NDCG@20': 0.8485771328157106, 'NDCG@30'